In [1]:
import pandas as pd
import numpy as np
import requests
import datetime
import plotly.express as px
import plotly.graph_objects as go
import scipy
import networkx as nx
import time

In [ ]:
def load_round_projects_data(round_id, chain_id):
    # prepare the URLs
    projects_url = 'https://indexer-grants-stack.gitcoin.co/data/' + chain_id +'/rounds/' + round_id + '/projects.json'
    print(projects_url)
    try:
        # download the Projects JSON data from the URL
        response = requests.get(projects_url)
        if response.status_code == 200:
            projects_data = response.json()

        # Extract the relevant data from each project
        projects = []
        for project in projects_data:
            project_data = {
                'id': project['id'],
                'title': project['metadata']['application']['project']['title'],
                'grantAddress': project['metadata']['application']['recipient'],
                'status': project['status'],
                'amountUSD': project['amountUSD'],
                'votes': project['votes'],
                'uniqueContributors': project['uniqueContributors'],
                'description': project['metadata']['application']['project']['description']
            }
            projects.append(project_data)
        # Create a DataFrame from the extracted data
        dfp = pd.DataFrame(projects)
        # Reorder the columns to match the desired order and rename column id to project_id
        dfp = dfp[['id', 'title', 'grantAddress', 'status', 'amountUSD', 'votes', 'uniqueContributors', 'description']]
        dfp = dfp.rename(columns={'id': 'project_id'})
        # Filter to only approved projects
        dfp = dfp[dfp['status'] == 'APPROVED']
        return dfp
    except:
        return pd.DataFrame()
    

def load_round_votes_data(round_id, chain_id):
    votes_url = 'https://indexer-grants-stack.gitcoin.co/data/' + chain_id + '/rounds/' + round_id + '/votes.json'
    try:
        # download the Votes JSON data from the URL
        response = requests.get(votes_url)
        if response.status_code == 200:
            votes_data = response.json()
        df = pd.DataFrame(votes_data)
        return df
    except:
        return pd.DataFrame()

def load_round_data(round_id, chain_id):
    dfp = load_round_projects_data(round_id, chain_id)
    dfv = load_round_votes_data(round_id, chain_id)
    dfv = pd.merge(dfv, dfp[['project_id', 'title', 'status']], how='left', left_on='projectId', right_on='project_id')
    return dfv

def load_passport_data():
    url = 'https://indexer-grants-stack.gitcoin.co/data/passport_scores.json'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            passport_data = response.json()
        passports = []
        for passport in passport_data:
            data = {
                'address': passport['address'],
                'last_score_timestamp': passport['last_score_timestamp'],
                'status': passport['status'],
                'rawScore': passport['evidence']['rawScore'] if 'evidence' in passport and passport['evidence'] is not None and 'rawScore' in passport['evidence'] else 0,
            }
            passports.append(data)
        df = pd.DataFrame(passports)
        df['last_score_timestamp'] = pd.to_datetime(df['last_score_timestamp'])
        return df
    except Exception as e:
        print('Error loading passport data')
        print(f'Error details: {e}')  # Print the details of the error
        return pd.DataFrame()


In [ ]:
round_id = '0x984e29dCB4286c2D9cbAA2c238AfDd8A191Eefbc'
chain_id = '10'
round_data = load_round_data(round_id, chain_id)
round_data.head(10)
df = round_data

In [ ]:
round_data.to_csv('citizens_round_votes.csv', index=False)

In [ ]:
df = pd.read_csv('citizens_round_votes.csv')
print(df.shape)

In [ ]:
dfpp = load_passport_data()
# keep passports that are in round
dfpp['address'] = dfpp['address'].str.lower()
dfpp = dfpp[dfpp['address'].isin(df['voter'].str.lower())]
# find the latest passport score for each address
dfpp = dfpp.sort_values(by=['address', 'last_score_timestamp'], ascending=False)
dfpp = dfpp.drop_duplicates(subset=['address'], keep='first')
print(dfpp.shape)
dfpp.head(10)

In [ ]:
dfpp.to_csv('citizens_round_passports.csv', index=False)

In [2]:
## START HERE

dfpp = pd.read_csv('citizens_round_passports.csv')
df = pd.read_csv('citizens_round_votes.csv')

print('Number of projects: ', df['project_id'].nunique())
print('Number of voters: ', df['voter'].nunique())
print('Total amountUSD: ', df['amountUSD'].sum())
print('Number of votes: ', df['amountUSD'].count())
print('Number of passports: ', dfpp['address'].nunique())

Number of projects:  33
Number of voters:  17021
Total amountUSD:  82968.02490585999
Number of votes:  57344
Number of passports:  12105


In [3]:
# merge dfpp with df
df['voter'] = df['voter'].str.lower()
df = pd.merge(df, dfpp[['address', 'rawScore']], how='left', left_on='voter', right_on='address')
df = df.rename(columns={'rawScore': 'passport_score'})
df['passport_score'] = df['passport_score'].fillna(0)
# convert passport_score to float
df['passport_score'] = df['passport_score'].astype(float)
df.head(10)

,id,transaction,blockNumber,projectId,applicationId,roundId,voter,grantAddress,token,amount,amountUSD,amountRoundToken,project_id,title,status,address,passport_score
0,0xa2ef81cd02b7de8739d384be71e0549c62368b737cd5...,0x4193c793a5a082818c36b4736f9a8c8eaabfa62bc0d0...,105529598,0x0ba24d30bb476404f6982bb4868cb351afb593fda0b4...,15,0x984e29dCB4286c2D9cbAA2c238AfDd8A191Eefbc,0x6a096fc442548dcbbfa3a0de4b01c125ba6b61ae,0x9DCbA70B2dfe5807e2A847E065EBb666791F8b8A,0x0000000000000000000000000000000000000000,10000000000000,0.017484,17487700000000000,0x0ba24d30bb476404f6982bb4868cb351afb593fda0b4...,Borderless,APPROVED,0x6a096fc442548dcbbfa3a0de4b01c125ba6b61ae,20.95
1,0xc671d80db54bb3d2fc456d6ae274a45416f957c6a1e1...,0x4193c793a5a082818c36b4736f9a8c8eaabfa62bc0d0...,105529598,0xe30120fba15bb05932c69f2518abef5548461ef6d62c...,51,0x984e29dCB4286c2D9cbAA2c238AfDd8A191Eefbc,0x6a096fc442548dcbbfa3a0de4b01c125ba6b61ae,0x3fB0D1e89693b8709de60d835452a4712d1c9B04,0x0000000000000000000000000000000000000000,10000000000000,0.017484,17487700000000000,0xe30120fba15bb05932c69f2518abef5548461ef6d62c...,Gitcoin Awareness and Female Founder Amplifica...,APPROVED,0x6a096fc442548dcbbfa3a0de4b01c125ba6b61ae,20.95
2,0xe175dba0fea13d8086261d2119781b3658b7f8561b0c...,0x4193c793a5a082818c36b4736f9a8c8eaabfa62bc0d0...,105529598,0x43a09a9e0a2c4c39ca4245a3c11fbd1d7def71275c93...,41,0x984e29dCB4286c2D9cbAA2c238AfDd8A191Eefbc,0x6a096fc442548dcbbfa3a0de4b01c125ba6b61ae,0xb62E762Af637b49Eb4870BCe8fE21bffF189e495,0x0000000000000000000000000000000000000000,10000000000000,0.017484,17487700000000000,0x43a09a9e0a2c4c39ca4245a3c11fbd1d7def71275c93...,ZER8's Gitcoin Citizen Round Application,APPROVED,0x6a096fc442548dcbbfa3a0de4b01c125ba6b61ae,20.95
3,0x32c19a76249aefe8d6300d0e69ab3a64570d8c98e035...,0x4193c793a5a082818c36b4736f9a8c8eaabfa62bc0d0...,105529598,0x9bc6d951ac597c8938efa6add91295a5ec90bc6c9197...,52,0x984e29dCB4286c2D9cbAA2c238AfDd8A191Eefbc,0x6a096fc442548dcbbfa3a0de4b01c125ba6b61ae,0xfC9265A28f66CF4561D74A4E25D7Bbd3F482B8e6,0x0000000000000000000000000000000000000000,10000000000000,0.017484,17487700000000000,0x9bc6d951ac597c8938efa6add91295a5ec90bc6c9197...,Jimi Cohen - Gitcoin Radio,APPROVED,0x6a096fc442548dcbbfa3a0de4b01c125ba6b61ae,20.95
4,0x47455e92cfd3a7523dcd54b0a0dff0cdfa96fa3c2188...,0x4193c793a5a082818c36b4736f9a8c8eaabfa62bc0d0...,105529598,0x82c03736f3abd9c21b31598ad17cb028b80876716727...,16,0x984e29dCB4286c2D9cbAA2c238AfDd8A191Eefbc,0x6a096fc442548dcbbfa3a0de4b01c125ba6b61ae,0xe1887fF140BfA9D3b45D0B2077b7471124acD242,0x0000000000000000000000000000000000000000,10000000000000,0.017484,17487700000000000,0x82c03736f3abd9c21b31598ad17cb028b80876716727...,Bankless Academy,APPROVED,0x6a096fc442548dcbbfa3a0de4b01c125ba6b61ae,20.95
5,0xd8b21fcda2b111f4bc3f7818593084414b71834f4c4d...,0x4193c793a5a082818c36b4736f9a8c8eaabfa62bc0d0...,105529598,0x5d79092725d85663a135c9841b6c8445f20132ee2595...,57,0x984e29dCB4286c2D9cbAA2c238AfDd8A191Eefbc,0x6a096fc442548dcbbfa3a0de4b01c125ba6b61ae,0xA40ba205ADD80C214eC7A710E790A54D738c4c27,0x0000000000000000000000000000000000000000,10000000000000,0.017484,17487700000000000,0x5d79092725d85663a135c9841b6c8445f20132ee2595...,40acres DAO,APPROVED,0x6a096fc442548dcbbfa3a0de4b01c125ba6b61ae,20.95
6,0xd840e75eb359eb8ec6ce4269f130c6109ce1dc1eab7a...,0x4193c793a5a082818c36b4736f9a8c8eaabfa62bc0d0...,105529598,0x7504217cb61d4b90163e8fb0f9801f597b6972a43ad7...,9,0x984e29dCB4286c2D9cbAA2c238AfDd8A191Eefbc,0x6a096fc442548dcbbfa3a0de4b01c125ba6b61ae,0x850a146D7478dAAa98Fc26Fd85e6A24e50846A9d,0x0000000000000000000000000000000000000000,10000000000000,0.017484,17487700000000000,0x7504217cb61d4b90163e8fb0f9801f597b6972a43ad7...,Zuzalu Gitcoin Hype Squad,APPROVED,0x6a096fc442548dcbbfa3a0de4b01c125ba6b61ae,20.95
7,0x02b2216d35c28b9fc4640a25bd8c2efeb50931ed7751...,0x4193c793a5a082818c36b4736f9a8c8eaabfa62bc0d0...,105529598,0x62974e6c4e31dfdaf58ccabea96cc589c38152acdb12...,34,0x984e29dCB4286c2D9cbAA2c238AfDd8A191Eefbc,0x6a096fc442548dcbbfa3a0de4b01c125

In [ ]:
print(df.columns)

In [4]:
# Calculate the number of unique voter addresses for each floor of raw score

df['floor_passport_score'] = df['passport_score'].apply(np.floor)
unique_voters = df.groupby('floor_passport_score')['voter'].nunique().reset_index()

# Create a histogram
fig1 = px.bar(unique_voters, x='floor_passport_score', y='voter', labels={'floor_passport_score': 'Passport Score', 'voter': 'Number of Unique Voter Addresses'}, title='Histogram of Passport Score by Number of Unique Voter Addresses')

# Calculate the percentage of users with a score above 15 and round it to one decimal place
percentage_above_15 = round((df[df['floor_passport_score'] >= 15]['voter'].nunique() / df['voter'].nunique()) * 100, 1)

# Add an annotation with the rounded percentage
fig1.add_annotation(
    x=15,
    y=unique_voters[unique_voters['floor_passport_score'] == 15]['voter'].values[0],
    text=f"{percentage_above_15}% of users have a score above 15",
    showarrow=True,
    arrowhead=1,
    ax=-40,
    ay=-50
)

# Display the histogram
fig1.show()

In [5]:
dff = pd.read_csv('features_citizens.csv')
print(dff.shape)
dff.head(10)

(57366, 36)


,block_timestamp,tx_hash,voter,project,amount_usd,__row_index,seed_same_naive,seed_same,seed_suspicious,less_5_tx,...,has_interaction_airdrop_m,has_interaction_tornado,has_interaction_disperse,flagged,stakeridoo_detected,doge_detected,odc_detected,has_seed_cluster,cluster_number,really_suspicicious_cluster
0,2023-06-27T19:28:25.000Z,0x11d201542132d813e3508936fc00e0a85a33e592bf4c...,0xff4997c57ca17ac3a481a7b88987e2341cdde61f,0xc98786d5a7a03c1e74affcb97ff7ef8a710da09b,0.996235,0,False,False,False,False,...,False,False,False,False,False,False,False,False,NaN,False
1,2023-06-26T23:41:01.000Z,0xcf39b3b5ad8be6aa97ba3f4407d5bdaeb5e38b3a4a1b...,0xc0e2412f1a3cfdbf638c55aebdebb58c9ae90a6a,0x745ce2af76e9a6eba65cc0cacaa9ea109bb7fabd,1.874705,1,False,True,True,False,...,False,False,False,False,False,False,False,False,NaN,False
2,2023-06-26T23:41:01.000Z,0xcf39b3b5ad8be6aa97ba3f4407d5bdaeb5e38b3a4a1b...,0xc0e2412f1a3cfdbf638c55aebdebb58c9ae90a6a,0xb62e762af637b49eb4870bce8fe21bfff189e495,1.874705,2,False,True,True,False,...,False,False,False,False,False,False,False,False,NaN,False
3,2023-06-26T23:50:05.000Z,0x69ec33f4e966af68d5cc0f29e5f58683a6367e91c03c...,0xabac490b9a2116d11c65edeb56af9a2c09e3ea56,0x8cfb71682feb93317d1eb4e0b3ca7fa9044169cf,1.124823,3,False,False,False,False,...,False,False,False,False,False,False,False,False,NaN,False
4,2023-06-26T23:36:07.000Z,0x849e0dd7c525ccb7a9b024b1e215464f39592fc054e3...,0xb36a9bb1bbc5262742d7b43d5f7284757de4637d,0x13257e783dc8ded7e227a28edb0428c42d31202e,1.031088,4,False,True,True,False,...,False,False,False,False,False,False,False,False,NaN,False
5,2023-06-26T23:55:27.000Z,0xfc1f7406608040b773a553dafd1ebf57279aa6a68892...,0x7f9a0df54ccf45546ddc170171f9580419ceb133,0xeb0cf83c80e4b4cd82196dac94e2c579672b6b1c,1.087329,5,False,True,True,False,...,False,False,False,False,False,False,False,False,NaN,False
6,2023-06-27T00:39:39.000Z,0x0b1d748a53d2c672afd5232ee088465082677adff673...,0xe1bfa9933cc2b55e3cddefab7b0f61d864d5162a,0x7904667c340601aab73939372c016dc5102732a2,1.015032,6,True,True,False,False,...,False,False,False,False,False,False,False,False,NaN,False
7,2023-06-27T00:17:15.000Z,0x70d2fc2970ff8a7d216ec4438138620754abdb974002...,0xdee2528b0612f91559abb862cbfb4741d9de2e10,0x521aacb43d89e1b8ffd64d9ef76b0a1074dedaf8,1.127814,7,False,False,False,False,...,False,False,False,False,False,False,False,False,NaN,False
8,2023-06-27T00:17:15.000Z,0x70d2fc2970ff8a7d216ec4438138620754abdb974002...,0xdee2528b0612f91559abb862cbfb4741d9de2e10,0xeb0cf83c80e4b4cd82196dac94e2c579672b6b1c,1.127814,8,False,False,False,False,...,False,False,False,False,False,False,False,False,NaN,False
9,2023-06-27T00:17:15.000Z,0x70d2fc2970ff8a7d216ec4438138620754abdb974002...,0xdee2528b0612f91559abb862cbfb4741d9de2e10,0x298f7f66ba43f0efecf4bc324b0016f822c783a4,1.127814,9,False,False,False,False,...,False,False,False,False,False,False,False,False,NaN,False


In [6]:
# join df and dff on tx_hash and transaction
dfm = pd.merge(dff, df[['voter', 'passport_score','floor_passport_score']], how='left', left_on='voter', right_on='voter')
print(dfm.shape)

(576846, 38)


In [ ]:
dfm.head(10)

In [7]:
# Calculate pass rates
def calculate_pass_rates(df):
    negative_indicators = ['has_lcs', 'has_interaction_toxic', 'has_interaction_disperse', 'is_airdrop_master', 'has_interaction_airdrop_m', 'interact_less_5tx', 'flagged', 'stakeridoo_detected', 'doge_detected', 'odc_detected', 'really_suspicicious_cluster']
    positive_indicators = []

    pass_rates = pd.DataFrame()

    # Calculate pass rates for negative indicators
    for indicator in negative_indicators:
        pass_rate = 1 - df.groupby(['voter', 'floor_passport_score'])[indicator].mean().groupby('floor_passport_score').mean()
        pass_rates = pd.concat([pass_rates, pass_rate.rename(indicator)], axis=1)

    # Calculate pass rates for positive indicators
    for indicator in positive_indicators:
        pass_rate = df.groupby(['voter', 'floor_passport_score'])[indicator].mean().groupby('floor_passport_score').mean()
        pass_rates = pd.concat([pass_rates, pass_rate.rename(indicator)], axis=1)

    return pass_rates.rename_axis('floor_passport_score').reset_index()

In [8]:
df_pass_rates = calculate_pass_rates(dfm)

In [ ]:
df_pass_rates.head(10)


In [9]:
# Calculate the number of unique voter addresses for each floor of raw score
unique_voters = df.groupby('floor_passport_score')['voter'].nunique().reset_index()

# Reshape the DataFrame from wide format to long format
pass_rates_long = df_pass_rates.melt(id_vars='floor_passport_score', var_name='indicator', value_name='not_sybil_rate')

# Merge unique voter counts with pass_rates_long DataFrame
pass_rates_long = pass_rates_long.merge(unique_voters, on='floor_passport_score', how='left')
# change name of voter to unique_voters
pass_rates_long.rename(columns={'voter': 'unique_voters'}, inplace=True)

# Create a scatter plot
fig2 = px.scatter(
    pass_rates_long,
    x='floor_passport_score',
    y='not_sybil_rate',
    color='indicator',
    hover_data=[pass_rates_long['unique_voters']],
    title='Not Sybil Rates by Floor of Raw Score for Each Indicator'
)

fig2.show()

In [10]:
# Calculate the Pearson correlation coefficient for each indicator
pearson_correlation_coefficients = pass_rates_long.groupby('indicator')[['floor_passport_score', 'not_sybil_rate']].corr().iloc[0::2, -1].reset_index()
pearson_correlation_coefficients = pearson_correlation_coefficients.rename(columns={'not_sybil_rate': 'Pearson Correlation Coefficient'})
pearson_correlation_coefficients = pearson_correlation_coefficients.sort_values(by='Pearson Correlation Coefficient', ascending=False)

# Calculate the Kendall correlation coefficient for each indicator
kendall_correlation_coefficients = pass_rates_long.groupby('indicator').apply(lambda x: x[['floor_passport_score', 'not_sybil_rate']].corr(method='kendall').iloc[0, 1]).reset_index()
kendall_correlation_coefficients = kendall_correlation_coefficients.rename(columns={0: 'Kendall Correlation Coefficient'})
kendall_correlation_coefficients = kendall_correlation_coefficients.sort_values(by='Kendall Correlation Coefficient', ascending=False)

# Merge the two dataframes on 'indicator'
correlation_coefficients = pd.merge(pearson_correlation_coefficients, kendall_correlation_coefficients, on='indicator')

# Display the merged table
display(correlation_coefficients[['indicator', 'Pearson Correlation Coefficient', 'Kendall Correlation Coefficient']])

,indicator,Pearson Correlation Coefficient,Kendall Correlation Coefficient
0,has_lcs,0.754421,0.712072
1,interact_less_5tx,0.360769,0.364483
2,really_suspicicious_cluster,0.303328,0.222815
3,has_interaction_disperse,0.284837,0.456835
4,doge_detected,-0.017403,0.007322
5,stakeridoo_detected,-0.028831,0.127885
6,has_interaction_airdrop_m,-0.345026,-0.242703
7,odc_detected,-0.633980,-0.515773
8,flagged,-0.667317,-0.523439
9,is_airdrop_master,-0.744424,-0.605866


In [ ]:
df.head(10)

In [11]:
# Create a new dataframe where each voter will be assigned a 0 or 1 for each of the 33 projects
df_pivot = df.pivot_table(index='voter', columns='title', values='amountUSD', aggfunc='size', fill_value=0)
# Turn the amounts into 1 (voted) or 0 (not voted)
df_pivot = df_pivot.applymap(lambda x: 1 if x > 0 else 0)
# Convert the rows into strings to create a voter_dna
df_pivot['voter_dna'] = df_pivot.apply(lambda row: ''.join(row.values.astype(str)), axis=1)

# Now calculate the counts for each 'voter_dna'
voter_dna_counts = df_pivot['voter_dna'].value_counts()

# Add this as a new column to the dataframe
df_pivot['dna_counts'] = df_pivot['voter_dna'].apply(lambda x: voter_dna_counts[x])
df_pivot.head(10)


title,40acres DAO,All for Climate DAO: Gitcoin support since GR12,Archimedes' Lever,Bankless Academy,Bankless Hindi,BanklessDAO's Gitcoin Citizens,Biteye,Blu3 Global,Bob Jiang,Borderless,...,Lefteris Karapetsas,OpenLore Library,Owocki/Supermodular.xyz (FBO Gitcoin Matching Pool),Quadratic Trust - Anne Connelly,ZER8's Gitcoin Citizen Round Application,Zuzalu Gitcoin Hype Squad,greenpill.network,🐙 Mars - Gitcoin citizen,voter_dna,dna_counts
voter,,,,,,,,,,,,,,,,,,,,,
0x000000006f457c0f8f560333d9c2877287d92a92,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,000000000000000000000001000000000,1410
0x000128fa45d79dc9af8016da242781f12c363fd5,0,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,000101100000110000000001000000000,6
0x000569d03483abb8949f729908465e65c4bdb243,0,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,000101100000100000000001000000000,631
0x0015f1d02b55ec9edd7536ca628fd30361252ebf,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,000001000000000000000000000000000,876
0x0015f91e93996d7e5979bf32391a8be59aadd606,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,000000000000000101000001000001001,1
0x0017f7adf1b404298efa3015ca0b56961fea4c34,0,0,1,0,0,1,0,1,0,1,...,0,0,0,0,1,0,1,0,001001010100101001000001100001010,1
0x001eabec5ae0e4dc40aecd9a75a6377fa0b31dcb,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,000100000000000000000000000000000,864
0x00207f5ccddfbe98b38c87cd6d39674cb0090c62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,000000000000100000000000000000000,294
0x002738a4a86d2591794fd36c120b36bad33d5f41,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,000001100000100000000001000000000,11


In [ ]:
print(df_pivot.columns[:-2])

In [12]:
# Create a dictionary to store the sum of dna_counts for each project
project_dna_counts = {project: 0 for project in df_pivot.columns[:-2]}
for _, row in df_pivot.iterrows():
    dna = row['voter_dna']
    dna_count = row['dna_counts']
    for project, voted in row[:-2].items():
        if voted == 1:
            project_dna_counts[project] += dna_count


In [14]:
grants_color = 'blue'
voters_color = 'red'
line_color = '#008F11'

# Initialize a new Graph
B = nx.Graph()

# Create nodes with the bipartite attribute
B.add_nodes_from(df['voter'].unique(), bipartite=0, color=voters_color) 
B.add_nodes_from(df['title'].unique(), bipartite=1, color=grants_color) 



# Add edges with amountUSD as an attribute
for _, row in df.iterrows():
    B.add_edge(row['voter'], row['title'], amountUSD=row['amountUSD'])



# Compute the layout
current_time = time.time()
pos = nx.spring_layout(B, dim=3, k = .09, iterations=50)
new_time = time.time()

# Nodes
node_x = []
node_y = []
node_z = []
sizes = []
for node in B.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_z.append(len(B[node]))  # Z coordinate is the number of connections
    # If node is a voter_dna, its size is based on the log of dna_counts.
    # Otherwise, its size is based on the sum of adjacent dna_counts.
    if node in df_pivot['voter_dna'].values:
        count = df_pivot.loc[df_pivot['voter_dna'] == node, 'dna_counts'].iloc[0]
        sizes.append(np.log(count+1)*10)  # Add 1 to avoid taking log of zero
    elif node in project_dna_counts:
        sizes.append(np.log(project_dna_counts[node]+1)*10)  # Use the sum of adjacent dna_counts for project nodes

node_trace = go.Scatter3d(
    x=node_x, y=node_y, z=node_z,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        size=sizes,
        sizemode='diameter',
        sizeref=0.05,
        sizemin=4,
        line=dict(width=2))
)

# Edges
edge_trace = []
for edge in B.edges():
    x0, y0 = pos[edge[0]]
    z0 = len(B[edge[0]])
    x1, y1 = pos[edge[1]]
    z1 = len(B[edge[1]])
    edge_trace.append(
        go.Scatter3d(
            x=[x0, x1, None], y=[y0, y1, None], z=[z0, z1, None],
            mode='lines',
            line=dict(width=1, color='grey'),
            hoverinfo='none'
        )
    )

# Create the network graph
layout = go.Layout(
    title='3D Network graph of voter DNA and projects',
    width=1000,
    height=1000,
    showlegend=False,
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False)
    ),
    margin=dict(
        t=100
    ),
    hovermode='closest',
    annotations=[
        dict(
            showarrow=False,
            xref='paper',
            yref='paper',
            x=0,
            y=0,
            xanchor='left',
            yanchor='bottom',
            text='3D Network graph of voter DNA and projects'
        )
    ],
)

fig = go.Figure(data=edge_trace + [node_trace], layout=layout)
fig.show()

ValueError: too many values to unpack (expected 2)

In [ ]:
display(df_dna_counts.head(10))

In [ ]:
grants_color = 'blue'
voters_color = 'red'
line_color = '#008F11'

# Initialize a new Graph
B = nx.Graph()

# Create nodes with the bipartite attribute
B.add_nodes_from(df['voter'].unique(), bipartite=0, color=voters_color) 
B.add_nodes_from(df['title'].unique(), bipartite=1, color=grants_color) 



# Add edges with amountUSD as an attribute
for _, row in df.iterrows():
    B.add_edge(row['voter'], row['title'], amountUSD=row['amountUSD'])



# Compute the layout
current_time = time.time()
pos = nx.spring_layout(B, dim=3, k = .09, iterations=50)
new_time = time.time()


    
# Extract node information
node_x = [coord[0] for coord in pos.values()]
node_y = [coord[1] for coord in pos.values()]
node_z = [coord[2] for coord in pos.values()] # added z-coordinates for 3D
node_names = list(pos.keys())
# Compute the degrees of the nodes 
degrees = np.array([B.degree(node_name) for node_name in node_names])
# Apply the natural logarithm to the degrees 
log_degrees = np.log(degrees + 1)
node_sizes = log_degrees * 10

# Extract edge information
edge_x = []
edge_y = []
edge_z = []  
edge_weights = []

for edge in B.edges(data=True):
    x0, y0, z0 = pos[edge[0]]
    x1, y1, z1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    edge_z.extend([z0, z1, None])  
    edge_weights.append(edge[2]['amountUSD'])

# Create the edge traces
edge_trace = go.Scatter3d(
    x=edge_x, y=edge_y, z=edge_z, 
    line=dict(width=1, color=line_color),
    hoverinfo='none',
    mode='lines',
    marker=dict(opacity=0.5))


# Create the node traces
node_trace = go.Scatter3d(
    x=node_x, y=node_y, z=node_z,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        color=[data['color'] for _, data in B.nodes(data=True)],  # color is now assigned based on node data
        size=node_sizes,
        opacity=1,
        sizemode='diameter'
    ))


node_adjacencies = []
for node, adjacencies in enumerate(B.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
node_trace.marker.color = [data[1]['color'] for data in B.nodes(data=True)]


# Prepare text information for hovering
node_trace.text = [f'{name}: {adj} connections' for name, adj in zip(node_names, node_adjacencies)]

# Create the figure
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='3D Network graph of voters and grants',
                    titlefont=dict(size=20),
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    annotations=[ dict(
                        showarrow=False,
                        text="This graph shows the connections between voters and grants based on donation data.",
                        xref="paper",
                        yref="paper",
                        x=0.005,
                        y=-0.002 )],
                    scene = dict(
                        xaxis_title='X Axis',
                        yaxis_title='Y Axis',
                        zaxis_title='Z Axis')))
                        
fig.show()